In [1]:
import pandas as pd
from datetime import datetime
from ics import Calendar, Event
from zoneinfo import ZoneInfo

In [7]:
df = pd.read_excel("lab_meeting_calendar.xlsx", sheet_name="2025")
df = df[df["Cancelled"]==False]  #filtering for lab meetings that are not cancelled
df = df.fillna("")
df

,Date,Lab Meeting,Journal Club,Technique of the Week,Food (no nuts),Cancelled,Notes
0,2025-06-30,Guofu,,,,False,
2,2025-07-14,Josh,,,Guofu,False,Bastille Day! Celebrate democracy with a lab ...
3,2025-07-21,,Solomon,,,False,
4,2025-07-28,Joy,,,Solomon,False,Joy's practice SMART talk
7,2025-08-18,Daniel,,,Joy,False,
8,2025-08-25,Solomon,,,Daniel,False,
10,2025-09-08,Ben,,,Solomon,False,
11,2025-09-15,,Justin,,Ben,False,
12,2025-09-22,Guofu,,,Justin,False,
13,2025-09-29,Justin,,,Guofu,False,


In [4]:
# Making an event log dictionary to add events to
event_log = {
    "Subject": [],
    "Start Date": [],
    "Start Time": [],
    "End Date": [],
    "End Time": [],
    "Location": [],
    "Description": [],
    "All Day Event": [],
    "Private": []
}

# Looping over all entries in the excel doc
for index, row in df.iterrows():
    # Event descriptions
    lab_meeting = row["Lab Meeting"]
    jc = row["Journal Club"]
    totw = row["Technique of the Week"]
    food = row["Food (no nuts)"]
    notes = row["Notes"]
    desc = f"Lab Meeting: {lab_meeting}\nJournal Club: {jc}\nTechnique of the Week: {totw}\nFood (no nuts): {food}\n\nNotes: {notes}"
    #print(desc)
    
    # Adding items to the dictionary
    event_log["Subject"].append("Frankfort Lab Meeting")
    event_log["Start Date"].append(row["Date"])
    event_log["Start Time"].append(datetime.strptime('10am', '%I%p').time())
    event_log["End Date"].append(row["Date"])
    event_log["End Time"].append(datetime.strptime('12pm', '%I%p').time())
    event_log["Location"].append("4th floor conference room")
    event_log["Description"].append(desc)
    event_log["All Day Event"].append(False)
    event_log["Private"].append(False)
    
# Turning dictionary into dataframe
df_csv = pd.DataFrame.from_dict(event_log)
df_csv

,Subject,Start Date,Start Time,End Date,End Time,Location,Description,All Day Event,Private
0,Frankfort Lab Meeting,2025-06-30,10:00:00,2025-06-30,12:00:00,4th floor conference room,Lab Meeting: Guofu\nJournal Club: \nTechnique ...,False,False
1,Frankfort Lab Meeting,2025-07-14,10:00:00,2025-07-14,12:00:00,4th floor conference room,Lab Meeting: Josh\nJournal Club: \nTechnique o...,False,False
2,Frankfort Lab Meeting,2025-07-21,10:00:00,2025-07-21,12:00:00,4th floor conference room,Lab Meeting: \nJournal Club: Solomon\nTechniqu...,False,False
3,Frankfort Lab Meeting,2025-07-28,10:00:00,2025-07-28,12:00:00,4th floor conference room,Lab Meeting: Joy\nJournal Club: \nTechnique of...,False,False
4,Frankfort Lab Meeting,2025-08-18,10:00:00,2025-08-18,12:00:00,4th floor conference room,Lab Meeting: Daniel\nJournal Club: \nTechnique...,False,False
5,Frankfort Lab Meeting,2025-08-25,10:00:00,2025-08-25,12:00:00,4th floor conference room,Lab Meeting: Solomon\nJournal Club: \nTechniqu...,False,False
6,Frankfort Lab Meeting,2025-09-08,10:00:00,2025-09-08,12:00:00,4th floor conference room,Lab Meeting: Ben\nJournal Club: \nTechnique of...,False,False
7,Frankfort Lab Meeting,2025-09-15,10:00:00,2025-09-15,12:00:00,4th floor conference room,Lab Meeting: \nJournal Club: Justin\nTechnique...,False,False
8,Frankfort Lab Meeting,2025-09-22,10:00:00,2025-09-22,12:00:00,4th floor conference room,Lab Meeting: Guofu\nJournal Club: \nTechnique ...,False,False
9,Frankfort Lab Meeting,2025-09-29,10:00:00,2025-09-29,12:00:00,4th floor conference room,Lab Meeting: Justin\nJournal Club: \nTechnique...,False,False


In [5]:
# df to display on the website
df_web = df[["Date", "Lab Meeting", "Journal Club", "Food (no nuts)", "Notes"]]
df_web.columns = ["Date", "Lab Meeting", "Journal Club", "Food", "Notes"]
#df_web["Start Time"] = "10:00am"
#df_web["End Time"] = "12:00pm"
df_web = df_web[["Date", "Lab Meeting", "Journal Club", "Food", "Notes"]]
df_web

,Date,Lab Meeting,Journal Club,Food,Notes
0,2025-06-30,Guofu,,,
2,2025-07-14,Josh,,Guofu,Bastille Day! Celebrate democracy with a lab ...
3,2025-07-21,,Solomon,,
4,2025-07-28,Joy,,Solomon,Joy's practice SMART talk
7,2025-08-18,Daniel,,Joy,
8,2025-08-25,Solomon,,Daniel,
10,2025-09-08,Ben,,Solomon,
11,2025-09-15,,Justin,Ben,
12,2025-09-22,Guofu,,Justin,
13,2025-09-29,Justin,,Guofu,


In [6]:
# Create calendar
calendar = Calendar()

# Setting the time zone to central time
tz = ZoneInfo("America/Chicago")

# Loop over DataFrame rows
for index, row in df_csv.iterrows():
    event = Event()
    event.name = row["Subject"]
    
    # Combine date and time
    start_dt = datetime.combine(pd.to_datetime(row["Start Date"]).date(), row["Start Time"]).replace(tzinfo=tz)
    end_dt = datetime.combine(pd.to_datetime(row["End Date"]).date(), row["End Time"]).replace(tzinfo=tz)
    
    #print(start_dt)
    #print(end_dt)
    
    event.begin = start_dt
    event.end = end_dt
    event.location = row["Location"]
    event.description = row["Description"]
    
    calendar.events.add(event)

In [ ]:
# Write to .ics file for Outlook calendar
#with open("frankfort_lab_meetings.ics", "w") as f:
#    f.writelines(calendar)

In [ ]:
# Exporting to csv for importing into outlook
#df_csv.to_csv("lab_meeting_calendar.csv", index=False)

In [ ]:
# Exporting to html for use on the lab website
#df_web.to_html("lab_meeting_calendar.html", index=False, justify="left", border=0)